In [22]:
from datasets import load_dataset
from transformers import (
    ViltProcessor,
    DefaultDataCollator,
    ViltForQuestionAnswering,
    TrainingArguments,
    Trainer,
)
import torch
from PIL import Image

In [23]:
processor = ViltProcessor.from_pretrained("model/baseline20epoch/checkpoint-68000")
model = ViltForQuestionAnswering.from_pretrained("model/baseline20epoch/checkpoint-68000")

In [24]:
import os
from os import listdir
from os.path import isfile, join
import json

# prepare image + question
eval_dir = os.path.abspath("data/test_data/test_data/")
eval_data = {}
for it in listdir(eval_dir):
    eval_data.update({it: []})
    for f in listdir(join(eval_dir, it)):
        file = join(eval_dir, it, f)
        eval_data[it].append(file)

def get_data(idx):
    img_url = eval_data[idx][0]
    image = Image.open(img_url)
    text_url = eval_data[idx][1]
    textf = open(text_url)
    # returns JSON object as 
    # a dictionary
    data = json.load(textf)
    text = data["question"]
    return image, text


In [26]:
import csv

# # prepare inputs
result = {}
for id in eval_data:
    image, text = get_data(id)
    encoding = processor(image, text, return_tensors="pt")
    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    result.update({id: [model.config.id2label[idx], logits[0][idx].item()]})


with open('results/vilt.csv', 'w', newline='') as csvfile:
    fieldnames = ['ID', 'Label', 'conf_score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for idx in result:
        writer.writerow({'ID': idx, 'Label': result[idx][0], 'conf_score': result[idx][1]})

In [27]:
result = {}
def read_csv2(filename):
    file = open(filename)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    results = {}
    for row in csvreader:
        results.update({row[0]: [row[1],row[2]]}) 
    return results
    
result = read_csv2("./results/vilt.csv")

In [28]:
import csv
def read_csv(filename):
    file = open(filename)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    results = {}
    for row in csvreader:
        results.update({row[0]: row[1]}) 
    return results

In [29]:
def write2csv(result, output):
    import csv
    with open(output, 'w', newline='') as csvfile:
        fieldnames = ['ID', 'Label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for idx in result:
            writer.writerow({'ID': idx, 'Label': result[idx]})

In [30]:
blipAug2 = read_csv("./results/blipAug31.csv")
vilt= read_csv("./results/vilt.csv")


In [31]:
ensemble_result = {}

for id in eval_data:
    image, text = get_data(id)
    if text == "What number is shown?":
        ensemble_result.update({id: blipAug2[id]})
        # print(id, ": ", blipAug2[id])
    # elif text == "How many shapes are green?":
    #     ensemble_result.update({id: vilt[id]})
    else:
        if float(result[id][1]) >= 12.5:
            ensemble_result.update({id: vilt[id]})
        else:
            ensemble_result.update({id: blipAug2[id]})


In [32]:
write2csv(ensemble_result, "results/ensemble.csv")

In [19]:
result

{}